#Lab.09 / IBM3202 – Folding Simulations using Structure-Based Models

###Theoretical aspects

While conventional molecular dynamics (MD) are very useful for predicting the dynamic behavior of biomolecules or deepening our understanding of experimental observations, their high computational cost introduces **limitations to the time scales of the motions that can be explored** (typically in the order of **ns – μs**). In contrast, the biological processes of greater complexity such as **protein folding, conformational changes and biomolecular interactions**, are in the order of **ms – s**. These phenomena cannot be studied through traditional MD strategies, thus requiring the application of enhanced sampling techniques in order to reach biologically relevant time scales.

In this regard, **the energy landscape theory of protein folding** and the **principle of minimum frustration** have provided the basis for the definition of a simplified potential energy function that allows us to explore these processes. This framework established that **the robustness of protein folding during evolution is the result of the selection of sequences that accumulate interactions that mutually support the native state of a protein**, while avoiding the accumulation of conflicting (frustrating) interactions for the stability of this folded state.

Such interactions are readily **available in the native state resolved by X-ray crystallography or NMR** for a given protein and can be extracted using a distance-based criteria (**Fig. 1**). In a scenario where all the interactions present in the native state are attractive, the **folding energy landscape is funneled**. These theoretical concepts constitute the central argument for the generation and application of **Structure-Based Models** (SBM) in MD simulations.

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/smog_01.png'/>
<figcaption>FIGURE 1. Using a distance-based criteria to determine native contacts based on a proteins structure, here the ribosomal protein S6. The left panel shows the proximity of the nearest atomic contact for each residue pair up to a maximum of 1.5 nm, whereas the right panel shows the resulting coarse-grained native contact map upon filtering by a distance cutoff of 0.6 nm and a minimal sequence separation of 3 residues between interacting residues <br> Noel JK & Onuchic JN (2012) <i> Computational Modeling of Biological Systems, 31-54</i></figcaption></center>
</figure>

**SBM models explicitly include a native bias in the potential energy function**, by considering all contacts between spatially close residues **in the native structure** of a protein as **attractive interactions**, while all non-native interactions (i.e. those that are not obtained through the analysis of the native structure) are treated as repulsive. This condition is consistent with the notion of a sequence of a protein without highly frustrated interactions (i.e. a funneled folding landscape). In this context, both the folding mechanism and the energy barriers mainly result from the geometrical constraints imposed by the connectivity of the polypeptide chain and its packaging in three-dimensional space, or **topology**.

Moreover, SBM models also reduce the complexity of the biomolecular system by using a coarse-grained approximation, in which each residue is replaced by a bead centered at the alpha carbon coordinates. However, newer SBM models are also able to employ **all-atom representations**.

The potential energy function that defines the bonded and non-bonded interactions of the SBM models includes the information regarding the topology of the native structure as follows:

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/smog_02.png' />
</center>
</figure>

Where $\varepsilon_{r} = 100\varepsilon$, $\varepsilon_{\theta} = 20 \varepsilon$, $\varepsilon_{D} = \varepsilon$, $\varepsilon_{C} = \varepsilon_{NC} = \varepsilon = k_{B}T$, which is the product between the Boltzmann constant and the temperature in reduced units; $r_{0}$, $\theta_{0}$ y $F_{D}(\phi_{0})$ are the bonds, angles and dihedrals of the native structure and are maintained by harmonic constraints; $\sigma_{ij}$ is the distance between the residues i and j in the native state (calculated from the native structure) and $\sigma_{NC}$ = 4 Å.

While native contacts are treated as attractive interactions through a 12-10 Lennard-Jones (LJ) potential, non-native contacts are treated as repulsive.
In this way, each time the temperature of the simulation is lower than the **folding temperature**, $T_{F}$, the system will tend to the minimum energy, in this case defined as the structure of the native state.

##Experimental Overview

In this tutorial we will simulate the folding of a small SH3 protein domain from the PDB structure 1FMK using a coarse-grained SBM and GROMACS.

For this, we will first generate a coarse-grained SBM model using **SMOG2** and then we will prepare and run the simulation files in a **SBM-enhanced version of GROMACS**. Finally, we will analyze the results in terms of the structural and energetical fluctuations of the simulated system, with the hopes of determining its free energy landscape using the **Weighted Histogram Analysis Method (WHAM)**.

#Part 0. Downloading and Installing the required software

Before we start, **remember to start the hosted runtime** in Google Colab.

Then, we must install several pieces of software to perform this tutorial. Namely:
- **biopython** for manipulation of the PDB files
- **py3Dmol** for visualization of the protein structure.
- **udocker** for installing and running Docker containers
- **SMOG2** for generating our structure-based models
- **SBM-enhanced GROMACS** for preparing our MD system and performing our MD simulations.

For visualizing our MD trajectories, we will employ a web version of **NGLview**. This is due to the inability of Google Colab to handle a required python package for loading NGLview directly onto Google Colab. Hopefully this will change in the near future.

1. First, we will start by installing **udocker** to pull and run a Docker version of **SMOG2** onto Google Colab.


In [ ]:
#Downloading udocker and installing smog2
!pip install udocker
!udocker --allow-root install
!udocker --allow-root pull smogserver/smog2

In [ ]:
#Test the smog2 docker installation
!udocker --allow-root run --rm -v $(pwd):/workdir smogserver/smog2:stable smog2 --help

2. Then, we will set up our SBM-enhanced GROMACS on Google Colab, based on a previously compiled and installed GROMACS.

In [ ]:
# Download and unzip the compressed folder of SBM-enhanced GROMACS
!wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/software/gromacs_sbm.tar.gz
!tar xzf gromacs_sbm.tar.gz

3. Lastly, we will install biopython and py3Dmol

In [ ]:
!pip install biopython py3dmol

Once these software installation processes are completed, we are ready to perform our experiments

# Part I – Generate a coarse-grained SBM model using SMOG2

For this step, we need an initial set of atomic coordinates, usually coming from a protein structure downloaded from the Protein Data Bank. Once downloaded, these PDB files must be cleaned from water molecules (in the case of crystal structures) or a single model must be selected from the many solutions for a single set of chemical shifts (in the case of NMR structures). Lastly, these coordinates are processed using SMOG2 to generate our coarse-grained SBM, in which each residue is represented by a bead centered at the coordinates of its corresponding C${\alpha}$ atom (thus termed **CA model**).

1. We will first start by creating and accessing a folder for preparing our system (remember our previous tutorial?)

In [ ]:
#Let's make a folder first. We need to import the os and path library
import os
from pathlib import Path

#Then, we define the path of the folder we want to create.
#Notice that the HOME folder for a hosted runtime in colab is /content/
smogpath = Path("/content/prepare_1FMK/")

#Now, we create the folder using the os.mkdir() command
#The if conditional is just to check whether the folder already exists
#In which case, python returns an error
if os.path.exists(smogpath):
  print("path already exists")
if not os.path.exists(smogpath):
  os.mkdir(smogpath)
  print("path was succesfully created")

In [ ]:
#Changing directory using python
os.chdir(smogpath)

2. Then, we will download the solved structure of the full-length human tyrosine kinase (PDB ID 1FMK). This solved proteins structure contains several domains, but we are only interested in its SH3 domain. Thus, we will again use `Dice` from biopython to not only remove water molecules and ligands, but also to select the residues that correspond to the SH3 domain.

**QUESTION❓**: By looking at the scripts below, which residues comprise the SH3 domain of human tyrosine kinase?

In [ ]:
#Importing your PDB file using biopython
import os
from Bio.PDB import *
pdbid = ['1fmk']
pdbl = PDBList()
for s in pdbid:
  pdbl.retrieve_pdb_file(s, pdir='.', file_format ="pdb", overwrite=True)
  os.rename("pdb"+s+".ent", s+".pdb")

In [ ]:
#Here we set up a parser for our PDB
parser = PDBParser()
io=PDBIO()
structure = parser.get_structure('X', '1fmk.pdb')
#And here we remove hydrogens, waters and ligands using Dice
io.set_structure(structure)
sel = Dice.ChainSelector('A', 82, 145)
io.save("1fmk_clean.pdb", sel)
print("Your PDB was processed. Only the protein heavy atoms have been kept!")

3. Let's examine the structure in py3Dmol

In [ ]:
import py3Dmol
#First we assign the py3Dmol.view as view
view=py3Dmol.view()
#The following lines are used to add the addModel class
#to read the PDB files
view.addModel(open('1fmk_clean.pdb', 'r').read(),'pdb')
#Here we set the background color as white
view.setBackgroundColor('white')
#Here we set the visualization style and color
view.setStyle({'chain':'A'},{'cartoon': {'color':'spectrum'}})
#Here we center the molecule for its visualization
view.zoomTo()
#And we finally visualize the structures using the command below
view.show()

4. The PDB structures require some editing on Google Colab before being parameterized using SMOG2:

- If a **C-terminal end residue** is present, we must rename it by adding a **`T`** character right after the residue name, for all atoms comprised by such C-terminal residue. This is required for SMOG2 to recognize C-terminal ends
  - Example:
```
# Before editing
ATOM    504  N   ALA A 145...
# After editing
ATOM    504  N   ALATA 145...
```
- **Eliminate `TER`** lines from the final PDB, and **only use it as separators between chains in oligomeric proteins**.
  - Example:
```
# Before editing
ATOM    508  CB  ALA A 145...
TER     509      ALA A 145
END
# After editing
ATOM    508  CB  ALA A 145...
END
```

To do this, we can use the **smog_adjustPDB** that comes with **SMOG2** to fix your PDB file as follows:

In [ ]:
!udocker --allow-root run --rm -v $(pwd):/workdir smogserver/smog2:stable smog_adjustPDB -i 1fmk_clean.pdb -o 1fmk_adjusted.pdb

5. Once this is done, we can process our adjusted file in **SMOG2** as indicated below:

In [ ]:
!udocker --allow-root run --rm -v $(pwd):/workdir smogserver/smog2:stable smog2 -i 1fmk_adjusted.pdb -CA -dname SH3_smog

6. Examine the generated SBM files. The most relevant files are described in detail below (and you can read this while you run your simulations)

- **.gro coordinate file** – File compatible with GROMACS that contains the total number of atoms and the coordinates of each atom in {x, y, z}
<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/smog_03.png' />
</center>
</figure>

  (**QUESTION❓:**is there anything particular with this .gro file?)


- **.top topology file** – Contains the topology, or bonded & non-bonded interactions, of the system. All of these parameters are extracted from the native structure defined in the PDB file
<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/smog_04.png' />
</center>
</figure>

  - The atomic composition of the system is described in the `[atoms]` section
  - Covalent bonds are listed in the `[bonds]` section
  - Angles between 3 atoms connected by covalent bonds are listed in the `[angles]` section
  - Dihedral angles between 4 atoms connected by covalent bonds (and also of planar configurations for aromatic side chains) are listed in the `[dihedrals]` section.
  - Non-bonded native contacts are indicated in the `[pairs]` section.
  - Non-native non-bonded interactions are treated as repulsive by listing the native contacts in the `[exclusions]` section; in other words, what is not listed in this section is non-native.

- **.ndx index file** – Lists groups of atoms that constitute the various chains registered in the .pdb file (for example, two or more chains of an oligomer)

- **.contacts.CG contact map file** – Archive of 4 columns, in which columns 2 and 4 contain the numbers of atoms in contact in the native state, and columns 1 and 3 correspond to the identifiers of the groups in the .ndx file (i.e. the chain to which each atom belongs to).

7. Let's visualize our final structure!

In [ ]:
import py3Dmol
#First we assign the py3Dmol.view as view
view=py3Dmol.view()
#The following lines are used to add the addModel class
#to read the PDB files
view.addModel(open('SH3_smog.gro', 'r').read(),'gro')
#Here we set the background color as white
view.setBackgroundColor('white')
#Here we set the visualization style and color
view.setStyle({'sphere':{'colorscheme':{'prop':'serial','gradient':'sinebow','min':1,'max':64}}})
#Here we center the molecule for its visualization
view.zoomTo()
#And we finally visualize the structures using the command below
view.show()

As you can see here, our structure is now **coarse-grained**, with each residue represented by a single bead centered at the C${\alpha}$ of the starting all-atom PDB structure.

#Part II - Prepare and run the SBM folding simulations in GROMACS

Now, we are ready to perform our simulations using these SBM models.

1. We will start by creating a new folder for preparing and running our MD simulations, in which we will copy our SBM coordinate and topology file.

In [ ]:
#Defining a new folder for the MD simulations
mdpath = Path("/content/md_1FMK/")

#Now, we create the folder using the os.mkdir() command
#The if conditional is just to check whether the folder already exists
#In which case, python returns an error
if os.path.exists(mdpath):
  print("path already exists")
if not os.path.exists(mdpath):
  os.mkdir(mdpath)
  print("path was succesfully created")

In [ ]:
#Changing to our newly created directory and copying the .gro and .top files
os.chdir(mdpath)
from shutil import copyfile
copyfile(smogpath/'SH3_smog.gro', mdpath/'SH3_smog.gro')
copyfile(smogpath/'SH3_smog.top', mdpath/'SH3_smog.top')

2. For GROMACS, we will also need a **MD instruction file**, as in our previous tutorial. We will download such file from GitHub (**mdrun_CA_v5.mdp**), which contains the parameters of the MD simulations, such as the integrator of the position and velocity of the atoms for each simulation time; the integration time step; the total simulation time; the frequency of writing the output files; the threshold distances for the treatment of the native interactions; the simulation temperature; etc.

  **WARNING⚠️:** It is strongly recommended that you read the comments and instructions contained in this file, as it is substantially different from the instructions used in a conventional MD.

  **QUESTION❓:** What temperature are we using to run our simulations?

In [ ]:
!wget https://github.com/pb3lab/ibm3202/raw/master/files/mdrun_CA_v5.mdp

The most important parameters in this .mdp file are the **temperature of the simulation**; the **temperature for the initial velocities**; the distance cut-off for evaluating the interactions; and the call for **user-defined tables** for our non-bonded interactions (which is explained below).

3. Internally, GROMACS has the ability to employ Lennard-Jones (LJ) potentials of type 12-6, whereas the potential required for native contacts in SBM simulations using CA models correspond to LJ 12-10. Thus, we will download a Perl script to generate a **user-defined table** of LJ 12-10 tabulated potentials. The result of running this script is a **.xvg file** that contains the tabulated potential.

In [ ]:
%%bash
wget https://github.com/pb3lab/ibm3202/raw/master/files/maketable4.pl
perl maketable4.pl > table.xvg

4. Now that we have all of the files that we need, we will first make a folder named after our simulation temperature, in which we will run our simulations.

In [ ]:
#Creating a folder for running a simulation at a given temperature
temppath = Path("152")

#Now, we create the folder using the os.mkdir() command
#The if conditional is just to check whether the folder already exists
#In which case, python returns an error
if os.path.exists(temppath):
  print("path already exists")
if not os.path.exists(temppath):
  os.mkdir(temppath)
  print("path was succesfully created")

5. Lastly, we will prepare our **.tpr portable binary run input file for GROMACS** in this folder and run our simulation! Please note how we instruct GROMACS to use our custom table of LJ 12-10 tabulated potentials.

This simulation takes ~10 min. **Reduce the number of steps according to your needs for the purpose of these tutorial**

In [ ]:
#Changing to our new folder
os.chdir(temppath)

In [ ]:
#Preparing our binary run input file
%%bash
source /content/gromacs_sbm/bin/GMXRC
gmx grompp -f ../mdrun_CA_v5.mdp -c ../SH3_smog.gro -p ../SH3_smog.top -o run.tpr

In [ ]:
#Running our simulation
%%time
%%bash
source /content/gromacs_sbm/bin/GMXRC
gmx mdrun -s run.tpr -table ../table.xvg -tablep ../table.xvg -nt 2 -noddcheck

Once our simulation is done (~10 min), we are ready to analyze it!

**But what about the temperature??** Something to keep in mind is that this temperature does not represent typical physical units, but so-called reduced (i.e. Boltzmann weighted) units, were $T^* = \varepsilon = k_BT$. This is required for further energetic analysis of the simulations. To better understand these reduced units, please refer to the SMOG-server FAQ.

#Part III – Analyzing our folding simulations

Once our simulations are done, we can analyze how the protein structure and potential energy fluctuates along the trajectory.

1. First, we will use the GROMACS utilities to obtain the changes in RMSD and potential energy along the trajectory. We will extract the RMSD using the `rmsd` module and the potential energy using the `energy` module.
  
  Please note how we are giving the appropriate commands to GROMACS to obtain these results!

In [ ]:
%%bash
source /content/gromacs_sbm/bin/GMXRC
#Commands for RMSD
echo "0" > options
echo " " >> options
echo "0" >> options
echo " " >> options
#RMSD calculation
gmx rms -s ../SH3_smog.gro -f traj_comp.xtc -xvg none < options
#Commands for potential energy
echo "Potential" > options
echo " " >> options
#Potential energy extraction
gmx energy -f ener.edr -xvg none < options

2. A better metric for looking at the fluctuations of the structure of our protein during the simulation is through the calculation of the changes in native contacts along the trajectory. For this, we will use the `g_kuh` module that is exclusively available in the SBM-enhanced version of GROMACS.

  For `g_kuh` to run, we first need an **.ndx index file** that contains a title and is followed by the list of atom pairs in contact in the native state, as shown below:
```
[ contacts ]
atom_i atom_j
...
```
  We can generate this file by printing the columns 2 and 4 of the **.contacts.CG file** generated by SMOG2:

In [ ]:
f = open(mdpath/"contacts.ndx", "a")
f.write("[ contacts ]\n")
with open(smogpath/"SH3_smog.contacts.CG") as infile:
  for line in infile:
    if line.strip():
      cols = line.split()
      f.write(cols[1] + "\t" + cols[3] + "\n")
f.close()

3. Then, we can run `g_kuh` to calculate the number of native contacts for each coordinate saved during the MD simulation in the **.xtc compressed trajectory file**.

  In this case, the `-s` option indicates our initial protein structure from which the native contact distances will be calculated, the `-n` option indicates our contact file, the `-noabscut` and `-cut` options indicate that `-cut` is taken to be the maximum relative deviation from the native distance for a formed contact (here, 20% longer than the native distance) and `-noshortcut` specifies that only deviations to larger distances than the native value are considered

In [ ]:
%%bash
source /content/gromacs_sbm/bin/GMXRC
g_kuh -s ../SH3_smog.gro -f traj_comp.xtc -n ../contacts.ndx -noabscut -noshortcut -cut 0.2

4. Let's plot our results and see what happened during our simulation! We will plot first the change in native contacts (Q), and then the change in potential energy. You can check the change in RMSD yourself

In [ ]:
!paste rmsd.xvg qvals.out > data.txt
import matplotlib.pyplot as plt
import numpy as np

data = np.loadtxt('data.txt')

plt.title('Structural fluctuations of the system')
plt.xlabel('Time (tau)')
plt.ylabel('Q')
plt.plot(data[:,0], data[:,2], linestyle='solid', linewidth='2', color='red')
plt.show()

In [ ]:
data = np.loadtxt('energy.xvg')

plt.title('Potential energy fluctuations of the system')
plt.xlabel('Time (tau)')
plt.ylabel('Energy')
plt.plot(data[:,0], data[:,1], linestyle='solid', linewidth='2', color='red')
plt.show()

If the simulations were correctly set at the folding temperature for this protein, you should observe **both the native** (high Q, low energy) **and unfolded** (low Q, high energy) **states** of the protein in equivalent ratios, as well as several transitions between these states

5. To finalize, we will visualize our simulation. For this, we will use the `trjconv` module to extract only the protein from our system and convert our trajectory into a PDB file and then  download this new PDB file and load it onto [**NGLviewer**](http://nglviewer.org/ngl/) as a **trajectory** PDB file.

In [ ]:
%%bash
source /content/gromacs_sbm/bin/GMXRC
#This is a trick to provide interactive options to gmx
echo "Protein" > options
echo "Protein" >> options
echo " "
gmx trjconv -s run.tpr -f traj_comp.xtc -o traj.pdb < options

In [ ]:
#Downloading the trajectory PDB file
from google.colab import files
files.download("/content/md_1FMK/152/traj.pdb")

**And this is the end of the ninth tutorial!**

If you want to download your results, you can perform the following commands:

In [ ]:
os.chdir("/content/")
!zip -r smogmd.zip $mdpath
!zip -r smog1FMK.zip $smogpath
from google.colab import files
files.download("/content/smogmd.zip")
files.download("/content/smog1FMK.zip")

**HOMEWORK📚:** Once you got here, you can perform a few exercises!

1.	Run a few simulations at different temperatures below and above the one we used in this tutorial. What do you expect if you lower/increase the temperature of the simulation?
2.	Run longer simulations. The analysis of protein folding requires several transitions between the native and unfolded states and any intermediate state that can be present en-route of protein folding.

#Appendix – Determining the folding landscape of your protein

Typically, sufficient sampling is achieved through the execution of several long MD simulations using these SBM models at different temperatures above and below the folding temperature of the protein of interest.

Once this is achieved, the free energy barrier that separates the unfolded and folded states of your protein (as well as the presence of any intermediates) as a function of the native contacts (or any other reaction coordinate) can be determined using the **Weighted Histograms Analysis Method (WHAM)**. Briefly, this method consists in describing the density of the states of the system as a function of the energy and an appropriate reaction coordinate (such as the number of native contacts), through an approximation of histograms weighted by $k_BT$, where $k_B$ corresponds to the **Boltzmann constant**.

1. We first need to generate files containing the change in potential energy and in the number of native contacts (Q) along the trajectory in the first and second columns, respectively. For this, we can use the `paste` and `awk` commands of `bash`

In [ ]:
!paste energy.xvg qvals.out | awk '{print $2, $3}' > hist.152

2. We perform the same operation for all other temperatures, making sure to change the filename according to the temperature being use (the example above is named hist.152 as it corresponds to T = 152), and we save all of the generated files into a single folder (e.g. analysis). An example folder can be downloaded from GitHub:

In [ ]:
#Going back to the main folder
import os
os.chdir("/content/")
!wget https://github.com/pb3lab/ibm3202/raw/master/files/WHAMexample.tar.gz
!tar zxf WHAMexample.tar.gz

3. We also download a **WHAM.jar** java application to perform the WHAM analysis through the use of a configuration file, also included in the previously downloaded folder

In [ ]:
os.chdir("/content/WHAMexample")
!wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/software/WHAM.jar

The config file (**please review it**) contains the information about the temperature range for which the heat capacity and the free energy will be calculated and the binning or spacing of the reaction coordinates (WHAM is a histogram analysis, hence the use of energy and Q bins). Also, at the end of the configuration file, the number of files to be analyzed and the filenames to be read with their corresponding temperatures must be indicated in the format:

```
# Example
numfiles X #here, the number of files must be indicated
name hist.152 temp 152
```

4. Once you have modified this script by adding the number, names and corresponding temperatures of the data that you will analyze, you can run WHAM by executing the following command:


In [ ]:
!java -jar WHAM.jar --config config

5. If everything goes well, you should obtain a file called **cv**, which corresponds to the heat capacity change during the unfolding process, and a series of **freeXXXX** files that are the free energies as a function of the number of native contacts calculated at different temperatures (T = XXX.X). Let's plot these results!

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

data = np.loadtxt('cv')

plt.title('Heat capacity')
plt.xlabel('Temperature (reduced units)')
plt.ylabel('C$_V$')
plt.xlim(144, 160)
plt.plot(data[:,0], data[:,1], linestyle='solid', linewidth='2', color='red')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

data = np.loadtxt('free1505')

plt.title('Free energy landscape')
plt.xlabel('Q')
plt.ylabel('F(Q) [$k_BT$]')
plt.xlim(0, 182)
plt.ylim(0, 9)
plt.plot(data[:,0], data[:,1], linestyle='solid', linewidth='2', color='red')
plt.show()

If we were to analyze the simulations generated in this tutorial, the number of transitions observed would be too small and the convergence of these data would be questionable. However, the repetition of these simulations for longer times and the collection of data at various temperatures between $0.9 > T_{F} > 1.1$, where $T_{F}$ is the folding temperature (~151), would generate results such as the ones presented above.